In [ ]:
#default_exp read_data

# Read data from framework
> Reading data from the platform.

In [1]:
# export
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm_notebook
from whatsapp_analytics import translation
import pickle
from os import path
import parmap
import numpy as np
from langdetect import DetectorFactory
DetectorFactory.seed = 0
from langdetect import detect
import re
import uuid
from account import baseurl,name,password
import json
import random
import re
import tqdm
import glob

parent_path='../Data/New_Data_15-06-2020/'

In [2]:
import neptune
from apiconfig import  project_name,proxies,api_token
# The init() function called this way assumes that 
# NEPTUNE_API_TOKEN environment variable is defined.
neptune.init(project_name,api_token=api_token)
neptune.set_project(project_name)
# log some metrics

Project(Hatespeech-CNERG/Annotation-Database)

In [ ]:
neptune.create_experiment('Fear speech annotation',send_hardware_metrics=False,run_monitoring_thread=False)

In [3]:
if path.exists(parent_path+'round_info.pkl'):
    with open(parent_path+'round_info.pkl', 'rb') as f:
        round_info = pickle.load(f)
else:
    print("no_round_info")

In [4]:
round_info=1

In [4]:
from doccano_api_client import DoccanoClient

In [5]:
doccano_client = DoccanoClient(
    baseurl,name,password
)

# get basic information about the authorized user
r_me = doccano_client.get_me()

# print the details from the above query
print(r_me.json())


<Response [200]>
{'id': 1, 'username': 'Punyajoy', 'first_name': 'Punyajoy', 'last_name': 'Saha', 'email': 'punyajoysaha1998@gmail.com', 'is_superuser': True}


In [6]:
import json
projects=doccano_client.get_project_list().json()

In [11]:
with open('user_dict.json') as f:
    user_dict_keyname = json.load(f)

In [12]:
user_dict_keyname

{'Divyanshu': {'id': 41, 'count': 300},
 'Debayu': {'id': 44, 'count': 0},
 'Danish': {'id': 54, 'count': 0},
 'Sharat': {'id': 53, 'count': 300},
 'Kukkadapu': {'id': 50, 'count': 301},
 'Adarsh': {'id': 52, 'count': 0},
 'Yuvraj': {'id': 49, 'count': 300},
 'Vineeth': {'id': 51, 'count': 299},
 'Samanway': {'id': 45, 'count': 0}}

In [13]:
with open('../Data/New_Data_22-05-2020/data_id_to_uuid.pkl', 'rb') as f:
        mapping1 = pickle.load(f)
with open('../Data/New_Data_15-06-2020/data_id_to_uuid.pkl', 'rb') as f:
        mapping2 = pickle.load(f)
         

In [14]:
users_to_ignore = ['Danish','Samanway']

def return_dict_label(project_id):
    label_list=doccano_client.get_label_list(project_id).json()
    dict_label={}
    for label in label_list:
        dict_label[label['id']]=label['text']
    return dict_label

def get_data_from_user(username,projects,mapping):
    project_name='Annotation_'+username
    user_id=user_dict_keyname[username]['id']
    print(username)
    for ele in projects:
        if(ele['name']==project_name):
            id1=ele['id']
    labels_dict =  return_dict_label(id1)
    training_annotated=doccano_client.get_doc_download(id1)
    #print(training_annotated.text)
    text_data = '[' + re.sub(r'\}\n\{', '},{', training_annotated.text) + ']'
    text_data=json.loads(text_data,encoding='utf8')
    print_stats={'Not_done':0,'Wrong_annotation':0,'Completed':0,'Not_found':0}
    tuple_data=[]
    
    for ele in tqdm.tqdm(text_data):
        found_id=re.search(r'\$\$(.*?)\$\$', ele['text']).group(1)
        if(len(ele['annotations'])==1):
            if(ele['annotations'][0]['user']==user_id):
                try:
                    tuple_data.append((mapping[found_id],labels_dict[ele['annotations'][0]['label']],username))
                    print_stats['Completed']+=1
                except KeyError:
                    print_stats['Not_found']+=1 
            else:
                print_stats['Not_done']+=1 
        elif(len(ele['annotations'])==0):
            print_stats['Not_done']+=1
        else:
            print(found_id)
            print_stats['Wrong_annotation']+=1
    print(print_stats)
    return tuple_data,print_stats

In [15]:
dict_ids_annotation={}
dict_user_annotation={}

dict_stats={}
for user in user_dict_keyname.keys():
    if(user not in users_to_ignore):
        tuple_data,print_stats=get_data_from_user(user,projects,mapping2)
        for data in tuple_data:
            try:
                dict_ids_annotation[data[0]].append(data[1])
                dict_user_annotation[data[0]].append(data[2])
            except:
                dict_ids_annotation[data[0]]=[data[1]]
                dict_user_annotation[data[0]]=[data[2]]
        dict_stats[user]=print_stats

Divyanshu


100%|██████████| 3228/3228 [00:00<00:00, 68974.86it/s]


{'Not_done': 0, 'Wrong_annotation': 0, 'Completed': 2728, 'Not_found': 500}
Debayu


100%|██████████| 2657/2657 [00:00<00:00, 71338.37it/s]


{'Not_done': 0, 'Wrong_annotation': 0, 'Completed': 2405, 'Not_found': 252}
Sharat


...

{'Not_done': 271, 'Wrong_annotation': 0, 'Completed': 649, 'Not_found': 250}
Yuvraj


100%|██████████| 3229/3229 [00:00<00:00, 86063.11it/s]


{'Not_done': 210, 'Wrong_annotation': 0, 'Completed': 2527, 'Not_found': 492}
Vineeth


100%|██████████| 2082/2082 [00:00<00:00, 91782.36it/s]

{'Not_done': 0, 'Wrong_annotation': 0, 'Completed': 1581, 'Not_found': 501}


In [16]:
temp_stats=pd.DataFrame(dict_stats)
temp_stats

,Divyanshu,Debayu,Sharat,Kukkadapu,Adarsh,Yuvraj,Vineeth
Not_done,0,0,249,0,271,210,0
Wrong_annotation,0,0,0,0,0,0,0
Completed,2728,2405,1904,2727,649,2527,1581
Not_found,500,252,500,501,250,492,501


In [17]:
processed_df=pd.read_pickle(parent_path+'data_processed_for_annotation.pkl')

In [18]:
def uniqueIndexes(l):
    seen = set()
    res = []
    for i, n in enumerate(l):
        if n not in seen:
            res.append(i)
            seen.add(n)
    return res

In [19]:
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score

In [20]:
import math
def return_fleiss_kappa(df):
    dict_map={'Fear speech':1,'Normal':0,'Not_Sure':0}
    listM=[]
    df=df[~((df['Annotation1']=='No_Annotation') | (df['Annotation2']=='No_Annotation') | ( df['Annotation3']=='No_Annotation'))]
    
    for index,row in df.iterrows():
        annotation_list=[0,0]
        annotation_list[dict_map[row['Annotation1']]]+=1
        annotation_list[dict_map[row['Annotation2']]]+=1
        annotation_list[dict_map[row['Annotation3']]]+=1
        listM.append(annotation_list)
    temp=fleiss_kappa(listM)
    if(math.isnan(temp)):
        return 1.0
    else:
        return temp

    
def return_class_count(df):
    dict_map={'Fear speech':1,'Normal':0}
    fear_count=0
    normal_count=0
    for index,row in df.iterrows():
        annotation_list=[0,0]
        try:
            annotation_list[dict_map[row['Annotation1']]]+=1
            annotation_list[dict_map[row['Annotation2']]]+=1
            annotation_list[dict_map[row['Annotation3']]]+=1
        except:
            annotation_list[0]=3
            print(index)
        if(annotation_list[1]>=annotation_list[0]):
            fear_count+=1
        else:
            normal_count+=1
        
        #listM.append(annotation_list)
    return fear_count,normal_count
    
    
def return_cohen_kappa_onevs_all(df,username):
    pair_tuples=[]
    for index,row in df.iterrows():
        pair_tuples.append((row['Annotation1'],row['User1'],row['Annotation2'],row['User2']))
        pair_tuples.append((row['Annotation1'],row['User1'],row['Annotation3'],row['User3']))
        pair_tuples.append((row['Annotation3'],row['User3'],row['Annotation2'],row['User2']))
    df_user_pair=pd.DataFrame(pair_tuples,columns=['Annotation1','User1','Annotation2','User2'])
    df_user_pair=df_user_pair[(df_user_pair['User1']==username)|(df_user_pair['User2']==username)]
    if(len(df_user_pair)>0):
        avg_agreement=[]
        for user in user_dict_keyname:
            if(user!=username):
                temp=df_user_pair[(df_user_pair['User1']==user)|(df_user_pair['User2']==user)]
                user1=[]
                user2=[]
                for index,row in temp.iterrows():
                    if(row['User1']==username):
                        user1.append(row['Annotation1'])
                        user2.append(row['Annotation2'])
                    else:
                        user1.append(row['Annotation2'])
                        user2.append(row['Annotation1'])
                        
                if(len(user1)>0):
                    temp=cohen_kappa_score(user1, user2)
                    if(math.isnan(temp)):
                        temp=1
                    
                    avg_agreement.append(temp)
        
        return np.mean(avg_agreement),int(len(df_user_pair)/2)
    else:
        print("User has not annotated yet")
        return 0.0,0.0

In [21]:
round_to_consider=10
annotation_data='../Data/Annotations_to_send/'
fear_total=0
normal_total=0
fix_len=len(processed_df)
for round_info in range(6,round_to_consider+1):
    if round_info <3:
        sample_df = pd.read_pickle(annotation_data+'annotation_'+str(round_info)+'.pkl')
    elif round_info >=3:
        temp_orig = pd.read_pickle(annotation_data+'annotation_'+str(round_info)+'.pkl')
        orig_index_list=[int(ele) for ele in list(temp_orig['orig_index'].values)]
        print("hello")
#         list_indexes=[]
#         for key in tqdm.tqdm(dict_ids_annotation, total=len(dict_ids_annotation)):
#             list_indexes.append(key-len(processed_df))
        sample_df=processed_df[processed_df['orig_index'].isin(orig_index_list)]
        print(len(sample_df))
    else:
        print("No such file exist")
        continue
    sample_df['Annotation1']= ['No_Annotation']*len(sample_df)
    sample_df['Annotation2']= ['No_Annotation']*len(sample_df)
    sample_df['Annotation3']= ['No_Annotation']*len(sample_df)
    sample_df['User1']= ['No_User']*len(sample_df)
    sample_df['User2']= ['No_User']*len(sample_df)
    sample_df['User3']= ['No_User']*len(sample_df)
    
    for key in tqdm.tqdm(dict_ids_annotation,total=len(dict_ids_annotation)):
        try:
#             if(len(dict_ids_annotation[key])!=3):
#                 continue
            #print(dict_user_annotation)
            res = uniqueIndexes(dict_user_annotation[key])    
#             if(len(res) < 3):
#                 continue
            
            count=1
            for i in res:
                if(count>3):
                    break
                #print(key,dict_user_annotation[key])
                if(round_info<3):
                    sample_df.loc[key]['Annotation'+str(count)]=dict_ids_annotation[key][i]
                    sample_df.loc[key]['User'+str(count)] =dict_user_annotation[key][i]
                else:
                    key1=key-fix_len
                    sample_df.at[key1,'Annotation'+str(count)]=dict_ids_annotation[key][i]
                    sample_df.at[key1,'User'+str(count)]=dict_user_annotation[key][i]
                count+=1
        except KeyError or IndexError:
            print("key error or index error")
            pass
    sample_df=sample_df.dropna()
    annotated_df=sample_df[~((sample_df['Annotation1']=='No_Annotation') | (sample_df['Annotation2']=='No_Annotation') | ( sample_df['Annotation3']=='No_Annotation'))]
    annotation_data_receive='../Data/Annotations_received/'
    sample_df.to_pickle(annotation_data_receive+'annotation_recv_'+str(round_info)+'.pkl')
    
    if(len(annotated_df)==0):
        continue
    print("==========X=====================X======")
    current_fleiss=return_fleiss_kappa(annotated_df)
    print("fleiss kappa",current_fleiss)
#     if(len(annotated_df)==len(sample_df)):
#         neptune.log_metric('fleiss_current_round',current_fleiss)
    print("=====================================")
    total_annotations=0
    for user in user_dict_keyname:
        cohen_kappa,count_user=return_cohen_kappa_onevs_all(annotated_df,user)
        print(user+'_cohens_kappa',cohen_kappa)
        print(user+'_count_annotation',count_user)
        total_annotations+=count_user
#         if(len(annotated_df)==len(sample_df)):
#             neptune.log_metric(user+'_cohens_kappa',cohen_kappa)
#             neptune.log_metric(user+'_count_annotation',count_user)
    
    print("Annotations done:",total_annotations)
    print("Annotations LEFT:",(len(sample_df)*3-total_annotations))
    fear,normal=return_class_count(annotated_df)
    print("Fear speech:",fear,'   Normal:',normal)
    fear_total+=fear
    normal_total+=normal
#     if(len(annotated_df)==len(sample_df)):
#         neptune.log_metric('annotation_total',fear_total+normal_total)
#         neptune.log_metric('fear_speech_found',fear_total)
    print("==========X=====================X======")


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

hello
233


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/pandas/core/frame.py:3155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[index, col] = value
100%|██████████| 4448/4448 [00:04<00:00, 919.13it/s] 


==========X=====================X======
fleiss kappa 0.3839099859353024
Divyanshu_cohens_kappa 0.39338102739280234
Divyanshu_count_annotation 116
User has not annotated yet
Debayu_cohens_kappa 0.0
Debayu_count_annotation 0.0
User has not annotated yet
Danish_cohens_kappa 0.0
Danish_count_annotation 0.0
Sharat_cohens_kappa 0.410525046822381
Sharat_count_annotation 117
Kukkadapu_cohens_kappa 0.3680009335613725
Kukkadapu_count_annotation 116
Adarsh_cohens_kappa 0.27434770975155065
Adarsh_count_annotation 117
Yuvraj_cohens_kappa 0.4356927396082562
Yuvraj_count_annotation 116
Vineeth_cohens_kappa 0.2768137397636143
Vineeth_count_annotation 117
User has not annotated yet
Samanway_cohens_kappa 0.0
Samanway_count_annotation 0.0
Annotations done: 699.0
Annotations LEFT: 0.0
Fear speech: 69    Normal: 164
==========X=====================X======


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

...

==========X=====================X======
fleiss kappa 0.4007596718657193
Divyanshu_cohens_kappa 0.44633249898256394
Divyanshu_count_annotation 375
Debayu_cohens_kappa 0.3608266478784188
Debayu_count_annotation 375
User has not annotated yet
Danish_cohens_kappa 0.0
Danish_count_annotation 0.0
User has not annotated yet
Sharat_cohens_kappa 0.0
Sharat_count_annotation 0.0
Kukkadapu_cohens_kappa 0.4116781302080224
Kukkadapu_count_annotation 374
User has not annotated yet
Adarsh_cohens_kappa 0.0
Adarsh_count_annotation 0.0
Yuvraj_cohens_kappa 0.384599370779744
Yuvraj_count_annotation 376
User has not annotated yet
Vineeth_cohens_kappa 0.0
Vineeth_count_annotation 0.0
User has not annotated yet
Samanway_cohens_kappa 0.0
Samanway_count_annotation 0.0
Annotations done: 1500.0
Annotations LEFT: 0.0
Fear speech: 136    Normal: 364
==========X=====================X======
hello


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

500


100%|██████████| 4448/4448 [00:04<00:00, 957.67it/s] 


==========X=====================X======
fleiss kappa 0.2794663964876731
Divyanshu_cohens_kappa 0.2671411726594827
Divyanshu_count_annotation 176
User has not annotated yet
Debayu_cohens_kappa 0.0
Debayu_count_annotation 0.0
User has not annotated yet
Danish_cohens_kappa 0.0
Danish_count_annotation 0.0
Sharat_cohens_kappa 0.22586486003923006
Sharat_count_annotation 51
Kukkadapu_cohens_kappa 0.24472626942498238
Kukkadapu_count_annotation 175
User has not annotated yet
Adarsh_cohens_kappa 0.0
Adarsh_count_annotation 0.0
Yuvraj_cohens_kappa 0.4518105184435982
Yuvraj_count_annotation 180
Vineeth_cohens_kappa 0.20649620981083025
Vineeth_count_annotation 171
User has not annotated yet
Samanway_cohens_kappa 0.0
Samanway_count_annotation 0.0
Annotations done: 753.0
Annotations LEFT: 747.0
Fear speech: 51    Normal: 200
==========X=====================X======


### Fleiss Kappa till this round

In [29]:
import glob
annotation_data_receive='../Data/Annotations_received/*'
files=glob.glob(annotation_data_receive)

In [30]:
files

['../Data/Annotations_received/annotation_recv_6.pkl',
 '../Data/Annotations_received/annotation_recv_1.pkl',
 '../Data/Annotations_received/annotation_recv_8.pkl',
 '../Data/Annotations_received/annotation_recv_3.pkl',
 '../Data/Annotations_received/annotation_recv_2.pkl',
 '../Data/Annotations_received/annotation_recv_9.pkl',
 '../Data/Annotations_received/annotation_recv_5.pkl',
 '../Data/Annotations_received/annotation_recv_7.pkl',
 '../Data/Annotations_received/annotation_recv_0.pkl',
 '../Data/Annotations_received/annotation_recv_10.pkl',
 '../Data/Annotations_received/annotation_recv_4.pkl']

In [31]:
import pandas as pd

In [32]:
# d2=d2.drop(list(set(d1.index).intersection(set(d2.index))))
# d2.to_pickle(files[2])

In [36]:
files=np.sort(files)
files



array(['../Data/Annotations_received/annotation_recv_0.pkl',
       '../Data/Annotations_received/annotation_recv_1.pkl',
       '../Data/Annotations_received/annotation_recv_10.pkl',
       '../Data/Annotations_received/annotation_recv_2.pkl',
       '../Data/Annotations_received/annotation_recv_3.pkl',
       '../Data/Annotations_received/annotation_recv_4.pkl',
       '../Data/Annotations_received/annotation_recv_5.pkl',
       '../Data/Annotations_received/annotation_recv_6.pkl',
       '../Data/Annotations_received/annotation_recv_7.pkl',
       '../Data/Annotations_received/annotation_recv_8.pkl',
       '../Data/Annotations_received/annotation_recv_9.pkl'], dtype='<U51')

In [37]:
list_df=[]
for file in files:
    
    df=pd.read_pickle(file)
    temp=return_atleast_one_count(df)
    try:
        fleiss_till_now = return_fleiss_kappa(temp)
    except:
        fleiss_till_now=0
    print('Round',file)
    print('Fear speech',len(temp[temp['one_fear_speech']==1]))
    print('Normal',len(temp[temp['one_fear_speech']==0]))
    print('Not decided',len(temp[temp['one_fear_speech']==-1]))
    print('Fleiss Kappa',fleiss_till_now)
    list_df.append(temp)
    

total_df = pd.concat(list_df)
#total_df = total_df.loc[~total_df.index.duplicated(keep='first')]

Round ../Data/Annotations_received/annotation_recv_0.pkl
Fear speech 169
Normal 296
Not decided 0
Fleiss Kappa 1.0
Round ../Data/Annotations_received/annotation_recv_1.pkl
Fear speech 25
Normal 475
Not decided 0
Fleiss Kappa 0.3154593050286711
Round ../Data/Annotations_received/annotation_recv_10.pkl
Fear speech 89
Normal 358
Not decided 53
Fleiss Kappa 0.2794663964876731
Round ../Data/Annotations_received/annotation_recv_2.pkl
Fear speech 47
Normal 416
Not decided 0
Fleiss Kappa 0.3476565380927798
Round ../Data/Annotations_received/annotation_recv_3.pkl
Fear speech 125
Normal 491
Not decided 0
Fleiss Kappa 0.35999843038769436
Round ../Data/Annotations_received/annotation_recv_4.pkl
Fear speech 147
Normal 387
Not decided 1
Fleiss Kappa 0.2649419165383121
Round ../Data/Annotations_received/annotation_recv_5.pkl
Fear speech 242
Normal 610
Not decided 0
Fleiss Kappa 0.3580055056638725
Round ../Data/Annotations_received/annotation_recv_6.pkl
Fear speech 69
Normal 164
Not decided 0
Fleiss K

In [34]:
len(total_df)

5677

In [ ]:
#files
df=pd.read_pickle('../Data/Annotations_received/annotation_recv_9.pkl')
df[df['Annotation2']!="No_Annotation"]

In [ ]:
temp_orig = pd.read_pickle(annotation_data+'annotation_'+str(8)+'.pkl')
        

In [ ]:
temp_orig

In [ ]:
total_df[total_df['one_fear_speech']==0][['message_text']].sample(10).to_csv('10_normal.csv')

In [ ]:
index_pilot=[5585410.0,
247024.0,
692000.0,
4822751.0,
5322724.0,
5256635.0,
6061616.0,
4792247.0,
1307247.0,
5223302.0,
4053832.0,
2641753.0,
1493359.0,
5254564.0,
236647.0,]

df[df['orig_index'].isin(index_pilot)][['message_text','Annotation1','Annotation2','Annotation3','User1','User2','User3']].to_csv('pilot_recv.csv')

In [ ]:
pd.set_option('display.max_colwidth', 600)

last_round_df=list_df[4]
user_to_check='Vineeth'

sample_user=last_round_df[(last_round_df['User1']==user_to_check) | (last_round_df['User3']==user_to_check) | (last_round_df['User3']==user_to_check)]
sample_user[['message_text','Annotation1','Annotation2','Annotation3','User1','User2','User3']].sample(10)

In [ ]:
last_round_df=list_df[3]
user1='Divyanshu'
sample_new=last_round_df[~(last_round_df['User1'].isin([user1]))]
sample_new=sample_new[~(sample_new['User2'].isin([user1]))]
sample_new=sample_new[~(sample_new['User3'].isin([user1]))]


In [ ]:
sample_new

In [ ]:
if(len(annotated_df)==len(sample_df)):
    neptune.log_metric('fleiss_kappa_total',fleiss_till_now)
    neptune.log_metric('annotation_total',total_annotated)
    neptune.log_metric('fear_speech_found',fear)
    
else:
    print('fleiss_kappa_total',fleiss_till_now)
    print('annotation_total',total_annotated)
    print('fear speech',fear)

In [ ]:
sample_df

In [28]:
def return_atleast_one_count(df):
    dict_map={'Fear speech':1,'Normal':0, 'No_Annotation':2}
    fear_count=[]
    for index,row in df.iterrows():
        annotation_list=[0,0,0]
        annotation_list[dict_map[row['Annotation1']]]+=1
        annotation_list[dict_map[row['Annotation2']]]+=1
        annotation_list[dict_map[row['Annotation3']]]+=1
        if(annotation_list[1]>=2):
            fear_count.append(1)
        elif(annotation_list[0]>=2):
            fear_count.append(0)
        else:
            fear_count.append(-1)
        #listM.append(annotation_list)
    df['one_fear_speech']=fear_count
    #sample_df=df[df['one_fear_speech']==1]
    return df

In [ ]:
last_round_df=return_atleast_one_count(last_round_df)

In [ ]:
sample_agreement=last_round_df[last_round_df['one_fear_speech'].isin([0,1])]

In [ ]:
dict_disagreement_fear_speech={}
dict_disagreement_normal={}
list_minimum_user=[]

for index,row in tqdm_notebook(sample_agreement.iterrows(),total=len(sample_agreement)):
    list_annotation={'Fear speech':[],'Normal':[]}
    
    for i in range(1,4):
        try:
            temp=row['Annotation'+str(i)]
            list_annotation[temp].append(row['User'+str(i)])
        except:
            pass
    print(list_annotation)    
    if(len(list_annotation['Fear speech'])>len(list_annotation['Normal'])  and len(list_annotation['Normal'])==1):
        list_minimum_user.append(list_annotation['Normal'][0])
        try:
            dict_disagreement_normal[list_annotation['Normal'][0]]+=1
        except:
            dict_disagreement_normal[list_annotation['Normal'][0]]=1
    elif(len(list_annotation['Fear speech'])<len(list_annotation['Normal']) and len(list_annotation['Fear speech'])==1):
        list_minimum_user.append(list_annotation['Fear speech'][0])
        try:
            dict_disagreement_fear_speech[list_annotation['Fear speech'][0]]+=1
        except:
            dict_disagreement_fear_speech[list_annotation['Fear speech'][0]]=1
    
    else:
        list_minimum_user.append('None')
    
sample_agreement['minority']=list_minimum_user
    

### Check high mistake users

In [ ]:
pd.set_option('display.max_colwidth', 1000)

#last_round_df=list_df[4]
user_to_check='Yuvraj'
sample_user=sample_agreement[sample_agreement['minority']==user_to_check]
sample_user[['message_text','Annotation1','Annotation2','Annotation3','User1','User2','User3']].to_csv(user_to_check+'.csv')


In [ ]:
dict_disagreement_normal

In [ ]:
sample_df=return_atleast_one_count(total_df)

In [ ]:
sample_df.to_csv('atleast_one_fear.csv',index=False)

In [ ]:
import string

puncts=["-",".","''","``","'","|","​","!",","]

import re
import emoji
#import nltk
from cltk.tokenize.word import WordTokenizer
#from nltk.corpus import stopwords
from cltk.stop.classical_hindi.stops import STOPS_LIST
tok = WordTokenizer(language='multilingual')
## libraries that can be used
def preprocess(df,params=None):
    '''This function should implememnt a multi-lingual tokenizer '''
    '''input: a document / sentence , params is a dict of control sequence'''
    '''output: should return a token list for the entire document/sentence'''
    list_tokenized=[]
    for index,row in tqdm_notebook(df.iterrows(),total=len(df)):
        s = row['message_text']
        s = emoji.demojize(s)
        s = re.sub(r"http\S+",'', s)
        s = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "",s)
        s = re.sub(r"/-", " ",s)
        s = re.sub(r"#,?,\,", " ",s)
        s = re.sub(r"[:\*]"," ",s)
        s=re.sub('[' + re.escape(''.join(puncts)) + ']', '', s)
        s=s.lower()
        msg= tok.tokenize(s)
        list_tokenized.append(msg)
        
    df['tokenized']=list_tokenized
    return df
 

In [ ]:
sample_df=preprocess(sample_df,params=None)

In [ ]:
filename='../Data/Important_lexicon/'+'muslim'+'_keywords.txt'
keyword_text = open(filename, 'r')
keywords = [line.strip('\n').lower() for line in keyword_text.readlines()]
    

In [ ]:
fear_speech_keyword_count={}
normal_keyword_count={}

for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
    temp = row['tokenized']
    for keyword in set(temp):
        if(row['one_fear_speech']==1):
            try:
                fear_speech_keyword_count[keyword]+=1
            except KeyError:
                fear_speech_keyword_count[keyword]=1
        
        if(row['one_fear_speech']==0):
            try:
                normal_keyword_count[keyword]+=1
            except KeyError:
                normal_keyword_count[keyword]=1
                

In [ ]:
normal_keyword_count

In [ ]:
dict1={k: v for k, v in sorted(normal_keyword_count.items(), key=lambda item: item[1],reverse=True)}
dict2={k: v for k, v in sorted(fear_speech_keyword_count.items(), key=lambda item: item[1],reverse=True)}


In [ ]:
from collections import Counter
vocab=[]
for word,count in Counter(dict2).most_common(10000):
    try:
        temp=dict1[word]
        vocab.append(count)
    except:
        pass

In [ ]:
x_child = np.array([dict2[word] for word in vocab])
x_base = np.array([dict1[word] for word in vocab]) + 1.

In [ ]:
mu = np.log(x_base) - np.log(x_base.sum())

In [ ]:
eta = estimate(x_child,mu)

In [ ]:
import json

with open('normal_keyword_count.json', 'w') as fp:
    json.dump(dict1, fp,indent=4,ensure_ascii=False)

In [ ]:
def rank_order_difference(dict1,dict2,max_rank=40):
    tuples_A=sorted(dict1.items(), key=lambda item: item[1],reverse=True)
    tuples_B=sorted(dict2.items(), key=lambda item: item[1],reverse=True)
    
    dict_A={}
    count=0
    for ele in tuples_A:
        if(count<max_rank):
            count+=1
        dict_A[ele[0]]=count
    
    dict_B={}
    count=0
    for ele in tuples_B:
        if(count<max_rank):
            count+=1
        dict_B[ele[0]]=count
    
    rank_diff_A={}
    for key in dict_A.keys():
        try:
            rank_diff_A[key]=dict_B[key]-dict_A[key]
        except:
            rank_diff_A[key]=max_rank-dict_A[key]
    rank_diff_B={}
    for key in dict_B.keys():
        try:
            rank_diff_B[key]=dict_A[key]-dict_B[key]
        except:
            rank_diff_B[key]=max_rank-dict_B[key]
            
    return rank_diff_A,rank_diff_B 

In [ ]:
rank_diff_A, rank_diff_B=rank_order_difference(dict1,dict2,max_rank=40)

In [ ]:
list_hashtag=sorted(rank_diff_B.items(), key=lambda item: item[1],reverse=True)[0:30]
[ele[0] for ele in list_hashtag]